# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#Create-Valuation-Model"><span class="toc-item-num">3 - </span>Create Valuation Model</a></div><div class="lev1"><a href="#Import-Data"><span class="toc-item-num">4 - </span>Import Data</a></div><div class="lev1"><a href="#Set-Outputs-to-Display"><span class="toc-item-num">5 - </span>Set Outputs to Display</a></div><div class="lev1"><a href="#Run-Valuation-Model"><span class="toc-item-num">6 - </span>Run Valuation Model</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import numpy
import pandas

from CorpFin.Valuation import UnlevValModel
from HelpyFuncs.Print import print_live_in_ipython

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [2]:
print_live_in_ipython()

# Set _CONSTANTS_

In [3]:
# Year 0
YEAR_0 = 2003

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 12

# Revenue Y-o-Y Growth Rates
REV_GROWTH_RATES = [numpy.nan, numpy.nan, .5, .4, .3, .2, .2, .2, .2, .2, .2, .2, .05]

# EBIAT / Revenue Ratios
EBIT_MARGINS = [numpy.nan, 0., .04 / .6, .05 / .6, .1, .1, .1, .1, .1, .1, .1, .1, .1]

# U.S. Corporate Tax rate
US_CORP_TAX_RATE = .4

# Further Investments & Working Capital Increases / Revenue Ratio
FIXED_ASSETS_OVER_REVENUE_RATIO = .1
NET_WORKING_CAPITAL_OVER_REVENUE_RATIO = -.6

# DISCOUNT RATE
DISCOUNT_RATE = .174

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .04

# Create Valuation Model

In [4]:
model = \
    UnlevValModel(
        venture_name='RightNowTechnologies',
        year_0=YEAR_0,
        nb_pro_forma_years_excl_0=NB_PRO_FORMA_YEARS)

Compiling:
    PublicMarketPremium... done after 0:00:00 (0:00:00 so far)
    ProFormaPeriodAssetDiscountRate... done after 0:00:00 (0:00:00 so far)
    ProFormaPeriodDiscountRate... done after 0:00:00 (0:00:00 so far)
    StabilizedDiscountRate... done after 0:00:00 (0:00:00 so far)
    Revenue... done after 0:00:00 (0:00:00 so far)
    RevenueChange... done after 0:00:00 (0:00:01 so far)
    RevenueGrowth... done after 0:00:01 (0:00:02 so far)
    OpEx... done after 0:00:00 (0:00:03 so far)
    OpEx_over_Revenue... done after 0:00:00 (0:00:04 so far)
    OpExGrowth... done after 0:00:00 (0:00:04 so far)
    EBIT... done after 0:00:01 (0:00:05 so far)
    EBITMargin... done after 0:00:01 (0:00:07 so far)
    EBITGrowth... done after 0:00:01 (0:00:08 so far)
    EBIAT... done after 0:00:02 (0:00:10 so far)
    FA... done after 0:00:02 (0:00:12 so far)
    FA_over_Revenue... done after 0:00:01 (0:00:14 so far)
    FAGrowth... done after 0:00:03 (0:00:17 so far)
    Depreciation... done 

# Import Data

In [5]:
data_df = \
    pandas.read_csv(
        'data/RightNow Technologies.csv',
        index_col='Year')
    
data_df.T

Year,2003,2004
Revenue,29300000,60000000


# Set Outputs to Display

In [6]:
outputs = \
    ['ProFormaPeriodDiscountRate', 'StabilizedDiscountRate',
     ' ',
     'REVENUE calcs:', 'Revenue', 'RevenueGrowth',
     '  ',
     'EBIT calcs:', 'EBIT', 'EBITMargin', 'EBITGrowth',
     '   ',
     'EBIAT calcs:', 'CorpTaxRate', 'EBIAT',
     '    ',
     'FIXED ASSETS & CAPEX calcs:', 'FA', 'FA_over_Revenue', 'CapEx',
     '     ',
     'NET WORKING CAPITAL calcs:', 'NWC', 'NWC_over_Revenue', 'NWCChange',
     '      ',
     'FCF & TV calcs:', 'FCF', 'LongTermGrowthRate',
     'TV', 'TV_RevenueMultiple', 'TV_EBITMultiple',
     '       ',
     'VALUATION calcs:', 'Unlev_Val']

# Run Valuation Model

In [7]:
results = \
    model(
        outputs=outputs,
        Revenue=(data_df.Revenue.astype(numpy.float32) / 1e6).tolist(),
        RevenueGrowth=REV_GROWTH_RATES,
        EBITMargin=EBIT_MARGINS,
        CorpTaxRate=US_CORP_TAX_RATE,
        FA_over_Revenue=(NB_PRO_FORMA_YEARS + 1) * [FIXED_ASSETS_OVER_REVENUE_RATIO],
        CapEx_over_Revenue=(NB_PRO_FORMA_YEARS + 1) * [0.],
        NWC=[-14.65],
        NWC_over_Revenue=(NB_PRO_FORMA_YEARS + 1) * [NET_WORKING_CAPITAL_OVER_REVENUE_RATIO],
        ProFormaPeriodDiscountRate=DISCOUNT_RATE,
        StabilizedDiscountRate=DISCOUNT_RATE,
        LongTermGrowthRate=LONG_TERM_GROWTH_RATE)

pandas.options.display.float_format = '{:,.3f}'.format
results['data_frame'].T

Calculating:
    ProFormaPeriodDiscountRate
    StabilizedDiscountRate
    Revenue
    RevenueGrowth
    EBIT
    EBITMargin
    EBITGrowth
    EBIAT
    FA
    FA_over_Revenue
    CapEx
    NWC
    NWC_over_Revenue
    NWCChange
    FCF
    TV
    TV_RevenueMultiple
    TV_EBITMultiple
    Unlev_Val
done!


,Year 0,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
ProFormaPeriodDiscountRate,0.174,,,,,,,,,,,,
StabilizedDiscountRate,0.174,,,,,,,,,,,,
,,,,,,,,,,,,,
REVENUE calcs:,,,,,,,,,,,,,
Revenue,29.300,60.000,90.000,126.000,163.800,196.560,235.872,283.046,339.656,407.587,489.104,586.925,616.271
RevenueGrowth,NaN,1.048,0.500,0.400,0.300,0.200,0.200,0.200,0.200,0.200,0.200,0.200,0.050
,,,,,,,,,,,,,
EBIT calcs:,,,,,,,,,,,,,
EBIT,0.000,0.000,6.000,10.500,16.380,19.656,23.587,28.305,33.966,40.759,48.910,58.693,61.627
EBITMargin,0.000,0.000,0.067,0.083,0.100,0.100,0.100,0.100,0.100,0.100,0.100,0.100,0.100
